In [3]:
import ipywidgets as widgets
import numpy as np
import cv2
from IPython.display import display
import matplotlib.pyplot as plt

B = plt.imread('scotty.jpg');
B = np.flip(B,2)
B = B[:,:,1]

B = cv2.resize(B,(100, 100))

In [9]:
int_range = widgets.FloatSlider(value=0,min=0,max=2*np.pi, step=0.01)
int_range2 = widgets.FloatSlider(value=0,min=0,max=49, step=0.01)
output2 = widgets.Output()
output3 = widgets.Output()

x = np.fft.fftfreq(100);#np.linspace(-1,1,100)
y = np.fft.fftfreq(100);#np.linspace(-1,1,100)
xv, yv = np.meshgrid(x, y)

xv = np.fft.fftshift(xv)
yv = np.fft.fftshift(yv)

wi1 = widgets.Image(format='png', width=300, height=300)
wi2 = widgets.Image(format='png', width=300, height=300)
sidebyside = widgets.HBox([wi1, wi2])


def on_value_change(change):
    #with output2, output3:
        theta = int_range.value
        rho   = int_range2.value
        
        x = np.round(rho*np.cos(theta))
        y = np.round(rho*np.sin(theta))
        
        fC = np.zeros((100,100))
        fC[int(50),int(50)] = 2*100*100
        fC[int(50+x),int(50+y)] = 100*100
        fC[int(50-x),int(50-y)] = 100*100
        
        C = np.fft.ifft2(np.fft.ifftshift(fC))
        C = np.abs(C)
        
        success, frame_bytes = cv2.imencode('.jpeg', 50*C)
        wi1.value = frame_bytes.tobytes()
        success, frame_bytes = cv2.imencode('.jpeg', np.abs(fC))
        wi2.value = frame_bytes.tobytes()
        
int_range.observe(on_value_change, names='value')
int_range2.observe(on_value_change, names='value')

on_value_change(0)

#display(image)
display(sidebyside)
display(int_range, output2)
display(int_range2, output3)

FloatSlider(value=0.0, max=6.283185307179586, step=0.01)

Output()

FloatSlider(value=0.0, max=49.0, step=0.01)

Output()

In [6]:
int_range = widgets.FloatSlider(value=0,min=0,max=50, step=0.01)
int_range2 = widgets.FloatSlider(value=0,min=0,max=50, step=0.01)
output2 = widgets.Output()
output3 = widgets.Output()

x = np.fft.fftfreq(100);#np.linspace(-1,1,100)
y = np.fft.fftfreq(100);#np.linspace(-1,1,100)
xv, yv = np.meshgrid(x, y)

xv = np.fft.fftshift(xv)
yv = np.fft.fftshift(yv)

wi1 = widgets.Image(format='png', width=300, height=300)
wi2 = widgets.Image(format='png', width=300, height=300)
sidebyside = widgets.HBox([wi1, wi2])


def on_value_change(change):
    #with output2, output3:
        x = int_range.value
        
        C = np.zeros((100,100))
        C[int(50-x):int(50+x),int(50-x):int(50+x)] = 1
        
        fC = np.fft.fftshift(np.fft.fft2(C))
        fC = np.abs(fC)
        
        success, frame_bytes = cv2.imencode('.jpeg', 255*C)
        wi1.value = frame_bytes.tobytes()
        success, frame_bytes = cv2.imencode('.jpeg', 255*np.sqrt(np.abs(fC)/255))
        wi2.value = frame_bytes.tobytes()
        
int_range.observe(on_value_change, names='value')
int_range2.observe(on_value_change, names='value')

on_value_change(0)

#display(image)
display(sidebyside)
display(int_range, output2)
#display(int_range2, output3)

FloatSlider(value=0.0, max=50.0, step=0.01)

Output()

In [7]:
int_range = widgets.FloatSlider(value=0,min=0.1,max=1.44, step=0.01)
int_range2 = widgets.FloatSlider(value=0,min=0,max=50, step=0.01)
output2 = widgets.Output()
output3 = widgets.Output()

x = np.fft.fftfreq(100);#np.linspace(-1,1,100)
y = np.fft.fftfreq(100);#np.linspace(-1,1,100)
xv, yv = np.meshgrid(x, y)

xv = np.fft.fftshift(xv)
yv = np.fft.fftshift(yv)

wi1 = widgets.Image(format='png', width=300, height=300)
wi2 = widgets.Image(format='png', width=300, height=300)
sidebyside = widgets.HBox([wi1, wi2])


def on_value_change(change):
    #with output2, output3:
        sigma = int_range.value**2
        
        C = np.zeros((99,99))
        
        xv, yv = np.meshgrid(np.linspace(-1,1,99), np.linspace(-1,1,99))
        
        C = np.exp(-(xv**2 + yv**2)/(sigma**2))/sigma
        
        fC = np.fft.fftshift(np.fft.fft2(C))
        fC = np.abs(fC)
        
        success, frame_bytes = cv2.imencode('.jpeg', 255*C/C.max())
        wi1.value = frame_bytes.tobytes()
        success, frame_bytes = cv2.imencode('.jpeg', 255*fC/fC.max())
        wi2.value = frame_bytes.tobytes()
        
int_range.observe(on_value_change, names='value')
int_range2.observe(on_value_change, names='value')

on_value_change(0)

#display(image)
display(sidebyside)
display(int_range, output2)
#display(int_range2, output3)

FloatSlider(value=0.1, max=1.44, min=0.1, step=0.01)

Output()

In [10]:
int_range = widgets.FloatSlider(value=1.44/2,min=0,max=1.44/2, step=0.01)
int_range2 = widgets.FloatSlider(value=0,min=0,max=1, step=0.01)
output2 = widgets.Output()
output3 = widgets.Output()

x = np.fft.fftfreq(100);#np.linspace(-1,1,100)
y = np.fft.fftfreq(100);#np.linspace(-1,1,100)
xv, yv = np.meshgrid(x, y)

xv = np.fft.fftshift(xv)
yv = np.fft.fftshift(yv)

wi1 = widgets.Image(format='png', width=300, height=300)
wi2 = widgets.Image(format='png', width=300, height=300)
sidebyside = widgets.HBox([wi1, wi2])


def on_value_change(change):
    #with output2, output3:
        A = (np.sqrt(xv**2 + yv**2) < int_range.value) & (np.sqrt(xv**2 + yv**2) >= int_range2.value)
        A = 1*A.astype(float)
        
        fB = np.fft.fftshift(np.fft.fft2(B))
        fC = fB*A
        C = np.fft.ifft2(np.fft.ifftshift(fC))
        C = np.abs(C)
        
        success, frame_bytes = cv2.imencode('.jpeg', C)
        wi1.value = frame_bytes.tobytes()
        success, frame_bytes = cv2.imencode('.jpeg', np.abs(fC)/100)
        wi2.value = frame_bytes.tobytes()
        
int_range.observe(on_value_change, names='value')
int_range2.observe(on_value_change, names='value')

on_value_change(0)

#display(image)
display(sidebyside)
display(int_range, output2)
display(int_range2, output3)

FloatSlider(value=0.72, max=0.72, step=0.01)

Output()

FloatSlider(value=0.0, max=1.0, step=0.01)

Output()